<div style="
      position: absolute;
      top: 50%;
      left: 62%;
      transform: translate(-50%, -50%);
      color: white;
      font-size: 20px;
  ">
    Intialization
  </div> <img src= "assets/bichon.png" style="height:75px;width:75px"/>

In [1]:
#general
import pandas as pd
import numpy as np

#plotting graphs
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

<div style="
      position: absolute;
      top: 50%;
      left: 62%;
      transform: translate(-50%, -50%);
      color: white;
      font-size: 20px;
  ">
    Data Cleaning
  </div> <img src= "assets/bichon.png" style="height:75px;width:75px"/>

In [2]:
#import dataset
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Clustering Algorithm for Recommender/raw_dataset/Sleep_health_and_lifestyle_dataset.csv')

#check data type and overall data distribution in columns
df.info()
df.describe()

#standardization of column names for project processes
df.columns = df.columns.to_series().apply(lambda x: x.strip().lower().replace(' ','_'))

#delete duplicate records
df.drop_duplicates(inplace=True)

#check total of nulls in each column
df.isna().any() #1 column with null found

#dealing with null on "sleep_disorder" column
print(df['sleep_disorder'].isna().sum()) #219
df['sleep_disorder'] = df['sleep_disorder'].fillna('None') #fill None datatype with 'None' string
print(df['sleep_disorder'].isna().sum()) #verify

#separating blood_pressure into systolic and diastolic columns
df[['systolic_rate','diastolic_rate']] = df['blood_pressure'].str.split('/',expand=True)
df['systolic_rate'] = pd.to_numeric(df['systolic_rate'])
df['diastolic_rate'] = pd.to_numeric(df['diastolic_rate'])

#dealing with 'normal' and 'normal weight' labels in BMI category
df['bmi_category'] = df['bmi_category'].apply(lambda x:'Normal' if x=='Normal Weight' else x)
df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB
219
0


,person_id,gender,age,occupation,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,bmi_category,blood_pressure,heart_rate,daily_steps,sleep_disorder,systolic_rate,diastolic_rate
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None,126,83
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None,125,80
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None,125,80
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea,140,90
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95


<div style="
      position: absolute;
      top: 50%;
      left: 62%;
      transform: translate(-50%, -50%);
      color: white;
      font-size: 20px;
  ">
    EDA Analysis Section
  </div> <img src= "assets/bichon.png" style="height:75px;width:75px"/>

In [3]:
#correlation mapping between features 
correlation_df = df.copy()

#remove person_id column
correlation_df.drop(columns=['person_id'],inplace=True)

#convert text based values into numerical features
for col in correlation_df.select_dtypes(include=['object']).columns:
    correlation_df[col] = LabelEncoder().fit_transform(correlation_df[col].astype(str))

#since most features' values are evenly distributed, use 'Pearson'
corr_matrix = correlation_df.corr(method='pearson')
fig = px.imshow(corr_matrix,x=corr_matrix.columns,y=corr_matrix.index,text_auto=True,
                color_continuous_scale='viridis',aspect='auto',title='Correlation Analysis')
fig.update_layout(height=700,width=1000)
fig.show()

#finding pairs of correlation that exceeds 0.5 but not self-correlation
corr_pairs = corr_matrix.unstack()
strong_corr = corr_pairs[(corr_pairs > 0.5) & (corr_pairs < 1)]
strong_corr = strong_corr.sort_values(ascending=False)

#split into columns and remove duplicate feature combination
strong_corr = strong_corr.reset_index()
strong_corr.columns = ['Feature1','Feature2','Correlation']
strong_corr = strong_corr[strong_corr['Feature1'] < strong_corr['Feature2']]
#print(strong_corr)

#plotting strong correlations > 0.5
strong_corr['pair'] = strong_corr['Feature1'] + " - " + strong_corr['Feature2']
fig_bar = px.bar(
    strong_corr,
    x='pair',
    y='Correlation',
    text='Correlation',
    color='Correlation',
    color_continuous_scale='Greens'
)
fig_bar.update_layout(title='Strong Correlations (>0.5)',
                      xaxis_title='Feature Pairs',
                      yaxis_title='Correlation',
                      xaxis_tickangle=60)

#plotting overall correlation distribution among features
all_corr = corr_matrix.unstack().reset_index()
all_corr.columns = ['Feature1','Feature2','Correlation']
all_corr = all_corr[all_corr['Feature1'] != all_corr['Feature2']]

fig_hist = px.histogram(
    all_corr,
    x='Correlation',
    nbins=20,
    title='Distribution of All Correlations',
    color_discrete_sequence=['lightgreen']
)

fig_hist.show()
fig_bar.show()

#strong negative correlations
corr_pairs = corr_matrix.unstack()
neg_corr = corr_pairs[corr_pairs < -0.5]
neg_corr = neg_corr.sort_values(ascending=True)
neg_corr = neg_corr.reset_index()
neg_corr.columns = ['Feature1','Feature2','Correlation']
neg_corr = neg_corr[neg_corr['Feature1'] < neg_corr['Feature2']]
#print(neg_corr)

#plotting strong negative correlations < -0.5
neg_corr['pair'] = neg_corr['Feature1'] + " - " + neg_corr['Feature2']
fig_bar = px.bar(
    neg_corr,
    x='pair',
    y='Correlation',
    text='Correlation',
    color='Correlation',
    color_continuous_scale='Peach'
)
fig_bar.update_layout(title='Strong Negative Correlations (<-0.5)',
                      xaxis_title='Feature Pairs',
                      yaxis_title='Correlation',
                      xaxis_tickangle=60,
                      yaxis=dict(range=[0, -1]))



In [4]:
#analyzing some correlated feature pairings (0.5<case<0.7)
#case 1 : top 5 occupations with highest overweight count
def bmi_occupation(df):
    temp_df = pd.DataFrame()
    temp_df[['person_id','occupation','bmi_category']] = df[['person_id','occupation','bmi_category']]
    
    #finding case for top 5 occupations with highest overweight
    overweight_job = temp_df[temp_df['bmi_category']=='Overweight']
    overweight_job = overweight_job.groupby('occupation')['person_id'].count().nlargest(5).reset_index(name='count')
   
    #finding the overall distribution of BMIs in these occupations
    overall_job = temp_df[temp_df['occupation'].isin(overweight_job['occupation'])].groupby(['occupation','bmi_category'])['person_id'].count().reset_index(name='count')
    overall_job['percentage'] = overall_job.groupby('occupation')['count'].transform(lambda x: x/x.sum()*100)

    fig1 = px.bar(overweight_job,x='count',y='occupation',title='Top 5 Occupations with Highest No.of Overweight',orientation='h')

    fig2 = px.bar(overall_job, x='percentage', y='occupation', color='bmi_category',barmode='group',title='Overall BMI Distribution among Top Overweight Occupations',
                  orientation='h')
    
    fig3 = px.sunburst(overall_job,path=['occupation','bmi_category'],values='count',color='occupation',color_discrete_sequence=px.colors.qualitative.Plotly,
                       title='Overall BMI Distribution among Top Overweight Occupations (Sunburst)')
    fig3.update_traces(textinfo='label+percent entry', textfont_color='white')
    
    #showing graphs
    fig1.show()
    fig2.show()
    fig3.show()



bmi_occupation(df)

In [5]:
#case 2 : finding relationship of stress level and heart rate (with occupation)
def stress_heart_rate(df):
    
    temp_df = pd.DataFrame()
    temp_df[['person_id','occupation','stress_level','heart_rate']] = df[['person_id','occupation','stress_level','heart_rate']]

    #finding average stress level and heart rate in all occupations
    overall_stress = temp_df.groupby('occupation')['stress_level'].mean().reset_index(name='average')
    overall_stress['average'] = overall_stress['average'].apply(lambda x:round(x,2))
    overall_stress = overall_stress.sort_values(by='average',ascending=False)
    #print(overall_stress)
    overall_heart = temp_df.groupby('occupation')['heart_rate'].mean().reset_index(name='average')
    overall_heart['average'] = overall_heart['average'].apply(lambda x:round(x,2))
    overall_heart = overall_heart.sort_values(by='average',ascending=False)
    #print(overall_heart)

    #combine both averages into one dataframe (get only for count >= 10)
    combined = overall_stress.merge(overall_heart, on='occupation', suffixes=('_stress', '_heart'))
    counts = temp_df.groupby('occupation')['person_id'].count().reset_index(name='count')
    combined = combined.merge(counts, on='occupation')
    combined = combined[combined['count']>=10] 
    
    #plotting graphs
    fig1 = px.scatter(temp_df,x='stress_level',y='heart_rate',title='Relationship between Stress and Heart Rate',trendline='ols')
    fig1.update_traces(selector=dict(mode='lines'),line=dict(dash='dot',color='orange'))

    fig2 = px.scatter(combined, x='average_stress',  y='average_heart', size='count', text='occupation', title='Average Stress & Heart Rate by Occupation')
    fig2.update_traces(textposition='top center')

    fig3 = px.bar(combined, y='occupation', x=['average_stress', 'average_heart'], barmode='group', orientation='h', title='Average Stress & Heart Rate by Occupation',
                  labels={'value':'average_level'})

    #showing graphs
    fig1.show()
    fig2.show()
    fig3.show()

stress_heart_rate(df)


In [10]:
#case 3 : age group and occupations with blood pressure
def age_occupations(df):
    temp_df = pd.DataFrame()
    temp_df[['person_id','age','occupation','systolic_rate','diastolic_rate']] = df[['person_id','age','occupation','systolic_rate','diastolic_rate']]

    #age group section
    temp_df['age_group'] = temp_df['age'].map(lambda x:get_age_group(x))
    diastolic_df = temp_df.groupby('age_group')['diastolic_rate'].mean().reset_index(name='average')
    systolic_df = temp_df.groupby('age_group')['systolic_rate'].mean().reset_index(name='average')
    combined = systolic_df.merge(diastolic_df, on='age_group',suffixes=('_diastolic','_systolic'))
    labels_sort = ['Young Adult','Middle Age','Old Age']
    combined['age_group'] = pd.Categorical(combined['age_group'],categories=labels_sort,ordered=True)
    combined = combined.sort_values(by='age_group',ascending=False)

    #occupation section
    diastolic_df2 = temp_df.groupby('occupation')['diastolic_rate'].mean().reset_index(name='average')
    systolic_df2 = temp_df.groupby('occupation')['systolic_rate'].mean().reset_index(name='average')
    combined2 = systolic_df2.merge(diastolic_df2, on='occupation',suffixes=('_diastolic','_systolic'))
    counts = temp_df.groupby('occupation')['person_id'].count().reset_index(name='count')
    combined2 = combined2.merge(counts, on='occupation')
    combined2 = combined2[combined2['count']>10]
    #print(combined2)

    #plotting graphs for age group and blood pressure
    x = temp_df['systolic_rate']    #regression line for overall average between diastolic and systolic
    y = temp_df['diastolic_rate']   #trendline 'ols' creates lines for each color group, therefore require manual implementation
    m, b = np.polyfit(x, y, 1)

    fig1 = px.scatter(temp_df,x='systolic_rate',y='diastolic_rate',color='age_group',title='Relationship between Systolic and Diastolic Rate',
                      color_discrete_map={'Young Adult': "#af72da", 'Middle Age': "#523ce1",'Old Age': "#4dba4d"})
    fig1.add_trace(go.Scatter(x=x,y=m*x+b,mode='lines',line=dict(dash='dot', color='orange'),showlegend=False))

    fig1.update_traces(selector=dict(mode='markers'), marker=dict(size=8))
    line_traces = [t for t in fig1.data if t.mode == 'lines']   #place markers above lines
    marker_traces = [t for t in fig1.data if t.mode == 'markers']
    fig1.data = tuple(line_traces + marker_traces)

    fig2 = px.bar(combined,x=['average_diastolic','average_systolic'],y='age_group',barmode='group',orientation='h',
                  title='Average Systolic and Diastolic Rate in Age Groups',labels={'value':'average_level'})

    #plotting graphs for occupation and blood pressure
    fig3 = px.scatter(combined2,x='average_systolic',y='average_diastolic',size='count',color='occupation',hover_name='occupation',
                      title='Average Systolic & Diastolic Rate by Occupation')
    fig3.update_traces(marker=dict(opacity=0.5))
    
    fig4 = px.bar(combined2,x=['average_diastolic','average_systolic'],y='occupation',barmode='group',title='Average Diastolic & Systolic by Occupation',
                  labels={'value':'average_level'})

    fig1.show()
    fig2.show()
    fig3.show()     #'lawyer' and 'salesperson' data points are overlapping 
    fig4.show()


def get_age_group(age):
    if age >=20 and age <=30:
        return 'Young Adult'
    elif age >30 and age <=45:
        return 'Middle Age'
    else:
        return 'Old Age'
    


age_occupations(df)

In [11]:
#case 4 : age group with bmi catgories
def age_bmi(df):
    
    temp_df = pd.DataFrame()
    temp_df[['person_id','age','bmi_category']] = df[['person_id','age','bmi_category']]
    temp_df['age_group'] = temp_df['age'].map(lambda x:get_age_group(x))
    dist_df = temp_df.groupby(['age_group','bmi_category'])['person_id'].count().reset_index(name='count')
    
    #finding percentage of overweights in each age group
    total_counts = temp_df.groupby('age_group')['person_id'].count().reset_index(name='total')
    all_weights_df = dist_df.merge(total_counts,on='age_group')
    all_weights_df['percentage'] = round((all_weights_df['count'] / all_weights_df['total'])*100,2)
    overweight_df = all_weights_df[all_weights_df['bmi_category']=='Overweight']

    #add age group sequence for line graph
    labels_sort = ['Young Adult','Middle Age','Old Age']
    overweight_df['age_group'] = pd.Categorical(overweight_df['age_group'],categories=labels_sort,ordered=True)
    overweight_df = overweight_df.sort_values(by='age_group',ascending=True)

    #plotting graphs
    fig1 = px.sunburst(dist_df,path=['age_group','bmi_category'],color='age_group',values='count',color_discrete_sequence=px.colors.qualitative.Plotly,
                       title='Distribution of BMI among Age Groups')
    fig1.update_traces(textinfo='label+percent entry',textfont_color='white')

    fig2 = px.bar(dist_df,x='age_group',y='count',color='bmi_category',barmode='group', title='BMI Categories Frequency by Age Groups',text='count')
    fig2.update_traces(textposition='outside',textfont=dict(size=11))
    fig2.update_yaxes(range=[0, dist_df['count'].max() + 50])

    fig3 = px.line(overweight_df,x='age_group',y='percentage',title='Overweight BMI Trend by Age',markers=True)
    fig3.update_yaxes(range=[0,overweight_df['percentage'].max() +10])
    fig3.update_traces(marker=dict(color='darkblue', size=9),line=dict(dash='dot'))

    fig1.show()
    fig2.show()
    fig3.show()
    

age_bmi(df)

C:\Users\josep\AppData\Local\Temp\ipykernel_14080\3885186791.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<div style="
      position: absolute;
      top: 50%;
      left: 62%;
      transform: translate(-50%, -50%);
      color: white;
      font-size: 20px;
  ">
    Machine Learning Section (K-Means Clustering)
</div> <img src= "assets/bichon.png" style="height:75px;width:75px"/>

In [12]:
#define features for clustering
df['MAP'] = df['diastolic_rate'] + 1/3*(df['systolic_rate']-df['diastolic_rate'])     #combine into mean arterial pressure as 1 feature
le = LabelEncoder()
df['bmi_encoded'] = le.fit_transform(df['bmi_category'])    #0 - Normal, 1 - Obese, 2 - Overweight
features = ['age','quality_of_sleep','physical_activity_level','stress_level','heart_rate','daily_steps','MAP','bmi_encoded']
temp_df = df[features].copy()

#scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(temp_df)

#define number of clusters (elbow method)
inertia = []
K = range(1, 10)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

#elbow plot (to find the optimal k) - largest drop
elbow_df = pd.DataFrame({'k': list(K), 'inertia': inertia})
elbow_df['inertia_diff'] = elbow_df['inertia'].diff() * -1  #-1 for positive difference
print(elbow_df)

#plotting graph
fig = px.line(elbow_df,x='k',y='inertia',markers=True,title='Elbow Method for Optimal k')
fig.update_layout(xaxis_title='Number of Clusters (k)', yaxis_title='Inertia')
fig.show()



   k      inertia  inertia_diff
0  1  2992.000000           NaN
1  2  2347.142706    644.857294
2  3  1651.870993    695.271713
3  4  1231.678047    420.192946
4  5   895.894018    335.784029
5  6   770.061794    125.832223
6  7   590.909027    179.152767
7  8   492.249437     98.659590
8  9   431.640705     60.608732


In [13]:
#fit k-means
optimal_k = elbow_df['inertia_diff'].idxmax()  
optimal_k = elbow_df.loc[optimal_k, 'k']
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
temp_df['cluster'] = kmeans.fit_predict(X_scaled)

#inspect cluster profiles
cluster_summary = temp_df.groupby('cluster').mean()
print(cluster_summary)

#plotting graphs using two features on each graph ---bookmark---
feature_pairs = [
    ('MAP', 'quality_of_sleep'),    #lifestyle on blood pressure
    ('heart_rate', 'physical_activity_level'),     #lifestyle on heart health
    ('physical_activity_level','bmi_encoded'),     #physical activity demographics
    ('quality_of_sleep', 'physical_activity_level'),     #recovery on physical activity
    ('age','stress_level')      #demographic on health metric
]

for x_feat, y_feat in feature_pairs:
    fig = px.scatter(
        temp_df,
        x=x_feat,
        y=y_feat,
        color='cluster',
        hover_data=features,
        title=f'Clusters: {x_feat} vs {y_feat}'
    )
    fig.show()




               age  quality_of_sleep  physical_activity_level  stress_level  \
cluster                                                                       
0        39.971910          8.016854                63.455056      4.539326   
1        51.095745          7.404255                71.382979      5.053191   
2        37.833333          6.000000                40.441176      7.166667   

         heart_rate  daily_steps         MAP  bmi_encoded  
cluster                                                    
0         68.185393  7137.078652   94.837079     0.011236  
1         69.989362  7621.276596  108.500000     1.978723  
2         73.784314  5516.666667   98.552288     1.156863  


<div style="
      position: absolute;
      top: 50%;
      left: 62%;
      transform: translate(-50%, -50%);
      color: white;
      font-size: 20px;
  ">
    Diet/Food Recommender
</div> <img src= "assets/bichon.png" style="height:75px;width:75px"/>

In [ ]:
#based on the cluster mean table and the graphs,
#cluster 0 - normal BMI, moderate daily steps, moderate blood pressure & heart rate, low stress, high sleep, moderate age (healthy avg age group)
#cluster 1 - overweight BMI, high daily steps, higher blood pressure, moderate heart rate, moderate-lower stress, moderate-high sleep, high age (old age group with avg health)
#cluster 2 - obese BMi, low daily steps, moderate blood pressure & heart rate, high stress, low sleep, moderate age (unhealthy avg age group)

'''
The daily meal recommended is solely based on simple research by author and cannot be followed without consulation of medical professionals
'''

#create diet plans for each sluter
#young adult to middle age group, generally healthy
diet_plans_0 = {
    "high_bp_high_stress": (
        "Breakfast: oatmeal + banana + walnuts"
        "\nLunch: grilled salmon + brown rice + steamed broccoli"
        "\nSnack: greek yogurt + berries"
        "\nDinner: Stir-fried tofu + spinach + sweet potato"
    ),
    "low_activity_low_sleep": (
        "Breakfast: oatmeal + berries + nuts"
        "\nLunch: chicken/tofu bowl + brown rice + vegetables"
        "\nSnack: yogurt + banana"
        "\nDinner: tofu / white fish + vegetables"
    ),
    "normal": (
        "Breakfast: sandwich + eggs"
        "\nLunch: soup-based"
        "\nSnack: nuts + banana"
        "\nDinner:	pasta + vegetables"
    )
}
#older middle age to old age group 
diet_plans_1 = {
    "high_bp_low_activity": (
        "Breakfast: oatmeal + chopped apples + little cinnamon"
        "\nLunch: steamed fish + steamed broccoli and carrots + brown rice"
        "\nSnack: unsalted almonds + pear / orange"
        "\nDinner: vegetable soup + grilled tofu + sweet potato"
    ),
    "low_sleep_high_stress_fat": (
        "Breakfast: greek yogurt + berries + chia seeds"
        "\nLunch: grilled chicken / turkey + steamd green beans / zucchini / bell peppers + brown rice"
        "\nSnack: banana / kiwi + walnuts"
        "\nDinner: light vegetable stir-fry + tofu + whole-grain pasta / quinoa"
    ),
    "normal": (
        "Breakfast: whole-grain toast / multigrain porridge + scrambled eggs / soft-boiled egg"
        "\nLunch: grilled fish + steamed vegetables + brown rice"
        "\nSnack: unsalted nuts + fresh fruit"
        "\nDinner: broth-based stew + light stir-fried tofu / chicken + whole grains" 
    )
}
#young adult to middle age group, generally unhealthy
diet_plans_2 = {
    "high_bp_high_stress": (
        "Breakfast: oatmeal + sliced banana + chia seeds"
        "\nLunch: steamed fish + steamed broccoli and carrots + brown rice"
        "\nSnack: greek yogurt + berries +walnuts"
        "\nDinner: stir-fried tofu with spinach / mushrooms + sweet potato"
    ),
    "low_activity_low_sleep": (
        "Breakfast: whole-grain toast + scrambled eggs + spinach"
        "\nLunch: chicken / tofu bowl with brown rice + steamed vegetables"
        "\nSnack: banana + almonds"
        "\nDinner: grilled white fish + light vegetable soup"
    ),
    "low_steps_fat": (
        "Breakfast: greek yogurt + berries + flax seeds"
        "\nLunch: grilled chicken salad + leafy greens, tomato, cucumber + quinoa"
        "\nSnack: apple slices + almonds"
        "\nDinner: stir-fried vegetables + brown rice + herbal tea"
    ),
    "normal": (
        "Breakfast: 0atmeal + berries + nuts"
        "\nLunch: grilled fish + steamed vegetables + brown rice"
        "\nSnack: fresh fruits + nuts"
        "\nDinner: stir-fried meat + light stir_fried vegetables / tofu + brown rice" 
    )
}

#diet recommender function
def recommender_func(row):

    #healthy group
    if row['cluster'] == 0:
        if row['MAP'] > 100 and row['stress_level'] > 6:
            return diet_plans_0['high_bp_high_stress']
        elif row['physical_activity_level'] < 50 and row['quality_of_sleep'] < 7:
            return diet_plans_0['low_activity_low_sleep']
        
        else:
            return diet_plans_0['normal']
        
    #older avg age group
    elif row['cluster'] == 1:
        if row['MAP'] >= 100 and row['physical_activity_level'] < 50:
            return diet_plans_1['high_bp_low_activity']
        elif row['quality_of_sleep'] < 7 and row['stress_level'] > 6 and row['bmi_encoded'] in (1,2):
            return diet_plans_1['low_sleep_high_stress_fat']
        else:
            return diet_plans_1['normal']
    
    #unhealthy group
    else:
        if row['MAP'] > 100 and row['stress_level'] > 6:
            return diet_plans_2['high_bp_high_stress']
        elif row['physical_activity_level'] < 50 and row['quality_of_sleep'] < 7:
            return diet_plans_2['low_activity_low_sleep']
        elif row['daily_steps'] < 5000 and row['bmi_encoded'] in (1,2):
            return diet_plans_2['low_steps_fat']
        else:
            return diet_plans_2['normal']

#get example daily diet recommendation   
temp_df['example_recommendation'] = temp_df.apply(recommender_func,axis=1)

#save the features and recommendations file
try:
    temp_df.to_csv("text_recommendations.csv",index=False)
except Exception as e:
    print(f'Error occured : {e}')
else:
    print(f'File successfully created!')



File successfully created!
